In [1]:
import pandas as pd
import json
import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy import URL

C:\Users\BigJulius\AppData\Local\Temp\ipykernel_15820\3255632492.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
pd.options.display.max_columns = 100

In [3]:
username = 'root'
password='root'
host='localhost'
database='jmdb'

engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}/{database}")


In [4]:
df = pd.read_csv("emdb.csv",encoding="utf-16", sep = ";")

In [5]:
df = df.rename(columns = {v : v.strip() for v in df.columns})

In [6]:
print(df.columns)

Index(['Numéro', 'Titre', 'Titre alternatif', 'Année', 'Pays', 'Genre(s)',
       'Langues', 'Durée', 'Chemin de l'affiche', 'Réalisateur(s)',
       'Scénariste(s)', 'Compositeur(s)', 'Studio', 'Distribution',
       'Bande-annonce', 'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
       'Commentaires', 'Etiquettes', 'Support', 'Taille du fichier',
       'ID TheMovieDb', 'ID IMDb'],
      dtype='object')


# table movie , director et intermediaire

In [7]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [8]:
# Modifier le chemin de l'affiche pour ne conserver que la dernière partie
df_movie["Chemin de l'affiche"] = df_movie["Chemin de l'affiche"].astype(str).apply(lambda x: x.split('\\')[-1])

# Afficher le DataFrame résultant
print(df_movie.head())  # Affiche les premières lignes du DataFrame

   Numéro                                     Titre  \
0    3711  La Danseuse des Folies Ziegfeld (vostfr)   
1    3710                       Une chance sur deux   
2    3709                     Un moment d'égarement   
3    3708                           Un ange de trop   
4    3707                             Les Guetteurs   

                   Titre alternatif Année  Durée Chemin de l'affiche  \
0    Ziegfeld Girl (Titre original)  1941    132          003795.jpg   
1                               NaN  1998    110          003794.jpg   
2                               NaN  1977     81          003793.jpg   
3  Heart Condition (Titre original)  1990    100          003792.jpg   
4     The Watchers (Titre original)  2024    102          003791.jpg   

                         Bande-annonce  \
0  www.youtube.com/watch?v=bRYo5Tsj5ok   
1  www.youtube.com/watch?v=QWmvDFZg7j0   
2  www.youtube.com/watch?v=_HxcgcWv3Hg   
3  www.youtube.com/watch?v=U5_C6IrxLWY   
4  www.youtube.com/watch?v

In [9]:
list_director_movie = ["Réalisateur(s)", 'Numéro']
df_director_movie = df.copy()[list_director_movie]

In [10]:
# df_director_movie ["Réalisateur(s)"] = df_director_movie["Réalisateur(s)"].str.split(",")
df_director_movie["Réalisateur(s)"] = df_director_movie["Réalisateur(s)"].str.split(",\s*")

In [11]:
df_director_movie  = df_director_movie.explode("Réalisateur(s)")

In [12]:
df_director  = df_director_movie.explode("Réalisateur(s)").drop_duplicates("Réalisateur(s)").reset_index(drop=True).reset_index()
df_director = df_director.rename(columns={"index":"id"}).drop(columns="Numéro")

#.reset_index().rename(columns={"index":"id"})

In [13]:
df_intermediaire_movie_director = df_director.merge(df_director_movie, on = "Réalisateur(s)").drop(columns="Réalisateur(s)")

In [14]:
# Remplacer les valeurs NULL de la colonne 'id' par 0
df_director['id'] = df_director['id'].fillna(0)
# Convertir la colonne 'id' en type entier
df_director['id'] = df_director['id'].astype(int)

# Réorganiser les ID si nécessaire pour éviter les conflits
max_id = df_director['id'].max()
if max_id > 0:
    df_director['id'] = df_director['id'] + 1

In [15]:
display(df_director)

,id,Réalisateur(s)
0,1,Robert Z. Leonard
1,2,Busby Berkeley
2,3,Patrice Leconte
3,4,Claude Berri
4,5,James D. Parriott
...,...,...
1823,1824,Richard C. Sarafian
1824,1825,Chris Petit
1825,1826,Brett Ratner
1826,1827,Buster Keaton


In [16]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3711,La Danseuse des Folies Ziegfeld (vostfr),Ziegfeld Girl (Titre original),1941,132,003795.jpg,www.youtube.com/watch?v=bRYo5Tsj5ok,GREATEST MUSICAL EXTRAVAGANZA OF ALL TIME!,Trois jeunes filles en quête de gloire sont en...,[My Book]F:\#DIVX\DIVX MOVIEZ\MUSICAL\Ziegfeld...,Matroska (MKV),NaN,Fichier multimédia,1.60 GB,movie/32619,34415.0
1,3710,Une chance sur deux,NaN,1998,110,003794.jpg,www.youtube.com/watch?v=QWmvDFZg7j0,C'est l'un des deux...,Alice sort de prison après avoir purgé une pei...,[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\une chan...,AVI (Microsoft),NaN,Fichier multimédia,2.00 GB,movie/49938,119247.0
2,3709,Un moment d'égarement,NaN,1977,81,003793.jpg,www.youtube.com/watch?v=_HxcgcWv3Hg,NaN,Deux pères emmènent leurs filles âgées de 17 a...,[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\Un momen...,AVI (Microsoft),NaN,Fichier multimédia,696 MB,movie/74871,76849.0
3,3708,Un ange de trop,Heart Condition (Titre original),1990,100,003792.jpg,www.youtube.com/watch?v=U5_C6IrxLWY,A tough cop. A dead lawyer. Every partnership ...,"Se remettant d'un arrêt cardiaque, le détectiv...",[My Book]F:\#DIVX\DIVX MOVIEZ\MadMov\Un Ange D...,Matroska (MKV),NaN,Fichier multimédia,1.91 GB,movie/41817,99750.0
4,3707,Les Guetteurs,The Watchers (Titre original),2024,102,003791.jpg,www.youtube.com/watch?v=YWdWv2iuo8Q,Laissez-les vous voir,Cette forêt n'existe sur aucune carte. Toutes ...,[My Book]F:\#DIVX\DIVX MOVIEZ\MadMov\The Watch...,Matroska (MKV),NaN,Fichier multimédia,4.59 GB,movie/1086747,26736843.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3706,5,le mécano de la général,The General (Titre original),1926,79,000005.jpg,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3707,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,000004.jpg,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3708,3,Les feux de la rampe,Limelight (Titre original),1952,137,000003.jpg,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3709,2,Les temps modernes,Modern Times (Titre original),1936,87,000002.jpg,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [17]:
display(df_intermediaire_movie_director)

,id,Numéro
0,0,3711
1,1,3711
2,1,3694
3,1,3693
4,2,3710
...,...,...
4002,1823,20
4003,1824,18
4004,1825,15
4005,1826,5


In [18]:
# Trouver l'index de la ligne avec le 'Réalisateur(s)' NULL
null_director_index = df_director[df_director['Réalisateur(s)'].isnull()].index

# Si une ligne avec 'Réalisateur(s)' NULL est trouvée
if not null_director_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_director.loc[null_director_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_director['id'].max()
    if max_id > 0:
        df_director.loc[df_director.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_director.head())

# Enregistrer le DataFrame modifié dans la table 'director' de la base de données
df_director.to_sql("director", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_director = df_director.merge(df_director_movie, on="Réalisateur(s)").drop(columns="Réalisateur(s)")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_director.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_director.to_sql("movie_director", index=False, con=engine, if_exists="replace")

     id     Réalisateur(s)
0  1829  Robert Z. Leonard
1     2     Busby Berkeley
2     3    Patrice Leconte
3     4       Claude Berri
4     5  James D. Parriott
     id  Numéro
0  1829    3711
1     2    3711
2     2    3694
3     2    3693
4     3    3710


4007

In [19]:
# df_director.to_sql("director",index=False,con = engine,if_exists = "replace" )

In [20]:
# df_intermediaire_movie_director.to_sql("movie_director",index=False,con = engine,if_exists = "replace" )

In [21]:
df_movie.to_sql("movies",index=False,con = engine,if_exists = "replace" )

3711

# table movie, screenwriter et intermediaire

In [22]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [23]:
list_screenwriter_movie = ["Scénariste(s)", 'Numéro']
df_screenwriter_movie = df.copy()[list_screenwriter_movie]

In [24]:
df_screenwriter_movie ["Scénariste(s)"] = df_screenwriter_movie["Scénariste(s)"].str.split(",\s*")

In [25]:
df_screenwriter_movie  = df_screenwriter_movie.explode("Scénariste(s)")

In [26]:
df_screenwriter  = df_screenwriter_movie.explode("Scénariste(s)").drop_duplicates("Scénariste(s)").reset_index(drop=True).reset_index()
df_screenwriter = df_screenwriter.rename(columns={"index":"id"}).drop(columns="Numéro")

In [27]:
df_intermediaire_movie_screenwriter = df_screenwriter.merge(df_screenwriter_movie, on = "Scénariste(s)").drop(columns="Scénariste(s)")

In [28]:
display(df_screenwriter)

,id,Scénariste(s)
0,0,Sonya Levien
1,1,Marguerite Roberts
2,2,Patrick Dewolf
3,3,Patrice Leconte
4,4,Claude Berri
...,...,...
3985,3985,Ted Tally
3986,3986,David Veloz
3987,3987,Richard Rutowski
3988,3988,Buster Keaton


In [29]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3711,La Danseuse des Folies Ziegfeld (vostfr),Ziegfeld Girl (Titre original),1941,132,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=bRYo5Tsj5ok,GREATEST MUSICAL EXTRAVAGANZA OF ALL TIME!,Trois jeunes filles en quête de gloire sont en...,[My Book]F:\#DIVX\DIVX MOVIEZ\MUSICAL\Ziegfeld...,Matroska (MKV),NaN,Fichier multimédia,1.60 GB,movie/32619,34415.0
1,3710,Une chance sur deux,NaN,1998,110,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=QWmvDFZg7j0,C'est l'un des deux...,Alice sort de prison après avoir purgé une pei...,[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\une chan...,AVI (Microsoft),NaN,Fichier multimédia,2.00 GB,movie/49938,119247.0
2,3709,Un moment d'égarement,NaN,1977,81,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=_HxcgcWv3Hg,NaN,Deux pères emmènent leurs filles âgées de 17 a...,[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\Un momen...,AVI (Microsoft),NaN,Fichier multimédia,696 MB,movie/74871,76849.0
3,3708,Un ange de trop,Heart Condition (Titre original),1990,100,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=U5_C6IrxLWY,A tough cop. A dead lawyer. Every partnership ...,"Se remettant d'un arrêt cardiaque, le détectiv...",[My Book]F:\#DIVX\DIVX MOVIEZ\MadMov\Un Ange D...,Matroska (MKV),NaN,Fichier multimédia,1.91 GB,movie/41817,99750.0
4,3707,Les Guetteurs,The Watchers (Titre original),2024,102,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=YWdWv2iuo8Q,Laissez-les vous voir,Cette forêt n'existe sur aucune carte. Toutes ...,[My Book]F:\#DIVX\DIVX MOVIEZ\MadMov\The Watch...,Matroska (MKV),NaN,Fichier multimédia,4.59 GB,movie/1086747,26736843.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3706,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3707,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3708,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3709,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [30]:
display(df_intermediaire_movie_screenwriter)

,id,Numéro
0,0,3711
1,0,2274
2,1,3711
3,2,3710
4,2,2035
...,...,...
7186,3985,15
7187,3986,14
7188,3987,14
7189,3988,5


In [31]:
# Trouver l'index de la ligne avec le 'Scénariste(s)' NULL
null_screenwriter_index = df_screenwriter[df_screenwriter['Scénariste(s)'].isnull()].index

# Si une ligne avec 'Scénariste(s)' NULL est trouvée
if not null_screenwriter_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_screenwriter.loc[null_screenwriter_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_screenwriter['id'].max()
    if max_id > 0:
        df_screenwriter.loc[df_screenwriter.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_screenwriter.head())

# Enregistrer le DataFrame modifié dans la table 'screenwriter' de la base de données
df_screenwriter.to_sql("screenwriter", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_screenwriter = df_screenwriter.merge(df_screenwriter_movie, on="Scénariste(s)").drop(columns="Scénariste(s)")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_screenwriter.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_screenwriter.to_sql("movie_screenwriter", index=False, con=engine, if_exists="replace")


     id       Scénariste(s)
0  3990        Sonya Levien
1     1  Marguerite Roberts
2     2      Patrick Dewolf
3     3     Patrice Leconte
4     4        Claude Berri
     id  Numéro
0  3990    3711
1  3990    2274
2     1    3711
3     2    3710
4     2    2035


7191

In [32]:
# df_screenwriter.to_sql("screenwriter",index=False,con = engine,if_exists = "replace" )

In [33]:
# df_intermediaire_movie_screenwriter.to_sql("movie_screenwriter",index=False,con = engine,if_exists = "replace" )

# table movie, music et intermediaire

In [34]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [35]:
list_music_movie = ["Compositeur(s)", 'Numéro']
df_music_movie = df.copy()[list_music_movie]

In [36]:
df_music_movie ["Compositeur(s)"] = df_music_movie["Compositeur(s)"].str.split(",\s*")

In [37]:
df_music_movie  = df_music_movie.explode("Compositeur(s)")

In [38]:
df_music  = df_music_movie.explode("Compositeur(s)").drop_duplicates("Compositeur(s)").reset_index(drop=True).reset_index()
df_music = df_music.rename(columns={"index":"id"}).drop(columns="Numéro")

In [39]:
df_intermediaire_movie_music = df_music.merge(df_music_movie, on = "Compositeur(s)").drop(columns="Compositeur(s)")

In [40]:
display(df_music)

,id,Compositeur(s)
0,0,Herbert Stothart
1,1,NaN
2,2,Michel Stelio
3,3,Patrick Leonard
4,4,Abel Korzeniowski
...,...,...
1311,1311,Machine Head
1312,1312,Photek
1313,1313,John Frizzell
1314,1314,Leonard Cohen


In [41]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3711,La Danseuse des Folies Ziegfeld (vostfr),Ziegfeld Girl (Titre original),1941,132,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=bRYo5Tsj5ok,GREATEST MUSICAL EXTRAVAGANZA OF ALL TIME!,Trois jeunes filles en quête de gloire sont en...,[My Book]F:\#DIVX\DIVX MOVIEZ\MUSICAL\Ziegfeld...,Matroska (MKV),NaN,Fichier multimédia,1.60 GB,movie/32619,34415.0
1,3710,Une chance sur deux,NaN,1998,110,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=QWmvDFZg7j0,C'est l'un des deux...,Alice sort de prison après avoir purgé une pei...,[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\une chan...,AVI (Microsoft),NaN,Fichier multimédia,2.00 GB,movie/49938,119247.0
2,3709,Un moment d'égarement,NaN,1977,81,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=_HxcgcWv3Hg,NaN,Deux pères emmènent leurs filles âgées de 17 a...,[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\Un momen...,AVI (Microsoft),NaN,Fichier multimédia,696 MB,movie/74871,76849.0
3,3708,Un ange de trop,Heart Condition (Titre original),1990,100,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=U5_C6IrxLWY,A tough cop. A dead lawyer. Every partnership ...,"Se remettant d'un arrêt cardiaque, le détectiv...",[My Book]F:\#DIVX\DIVX MOVIEZ\MadMov\Un Ange D...,Matroska (MKV),NaN,Fichier multimédia,1.91 GB,movie/41817,99750.0
4,3707,Les Guetteurs,The Watchers (Titre original),2024,102,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=YWdWv2iuo8Q,Laissez-les vous voir,Cette forêt n'existe sur aucune carte. Toutes ...,[My Book]F:\#DIVX\DIVX MOVIEZ\MadMov\The Watch...,Matroska (MKV),NaN,Fichier multimédia,4.59 GB,movie/1086747,26736843.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3706,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3707,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3708,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3709,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [42]:
display(df_intermediaire_movie_music)

,id,Numéro
0,0,3711
1,0,2637
2,0,2503
3,1,3710
4,1,3705
...,...,...
4111,1311,46
4112,1312,46
4113,1313,42
4114,1314,14


In [43]:
# Trouver l'index de la ligne avec le 'Compositeur(s)' NULL
null_music_index = df_music[df_music['Compositeur(s)'].isnull()].index

# Si une ligne avec 'Compositeur(s)' NULL est trouvée
if not null_music_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_music.loc[null_music_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_music['id'].max()
    if max_id > 0:
        df_music.loc[df_music.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_music.head())

# Enregistrer le DataFrame modifié dans la table 'music' de la base de données
df_music.to_sql("music", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_music = df_music.merge(df_music_movie, on="Compositeur(s)").drop(columns="Compositeur(s)")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_music.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_music.to_sql("movie_music", index=False, con=engine, if_exists="replace")


     id     Compositeur(s)
0  1316   Herbert Stothart
1     0                NaN
2     2      Michel Stelio
3     3    Patrick Leonard
4     4  Abel Korzeniowski
     id  Numéro
0  1316    3711
1  1316    2637
2  1316    2503
3     0    3710
4     0    3705


4116

In [44]:
# df_music.to_sql("music",index=False,con = engine,if_exists = "replace" )

In [45]:
df_intermediaire_movie_music.to_sql("movie_music",index=False,con = engine,if_exists = "replace" )

4116

# table movie, studio et intermediaire

In [46]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [47]:
list_studio_movie = ["Studio", 'Numéro']
df_studio_movie = df.copy()[list_studio_movie]

In [48]:
df_studio_movie ["Studio"] = df_studio_movie["Studio"].str.split(",\s*")

In [49]:
df_studio_movie  = df_studio_movie.explode("Studio")

In [50]:
df_studio  = df_studio_movie.explode("Studio").drop_duplicates("Studio").reset_index(drop=True).reset_index()
df_studio = df_studio.rename(columns={"index":"id"}).drop(columns="Numéro")

In [51]:
df_intermediaire_movie_studio = df_studio.merge(df_studio_movie, on = "Studio").drop(columns="Studio")

In [52]:
display(df_studio)

,id,Studio
0,0,Metro-Goldwyn-Mayer
1,1,TF1 Films Production
2,2,Renn Productions
3,3,New Line Cinema
4,4,Jerry Bruckheimer Films
...,...,...
994,994,ADR Distribution
995,995,A-Film Distribution
996,996,Dybex
997,997,British Empire Films Australia


In [53]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3711,La Danseuse des Folies Ziegfeld (vostfr),Ziegfeld Girl (Titre original),1941,132,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=bRYo5Tsj5ok,GREATEST MUSICAL EXTRAVAGANZA OF ALL TIME!,Trois jeunes filles en quête de gloire sont en...,[My Book]F:\#DIVX\DIVX MOVIEZ\MUSICAL\Ziegfeld...,Matroska (MKV),NaN,Fichier multimédia,1.60 GB,movie/32619,34415.0
1,3710,Une chance sur deux,NaN,1998,110,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=QWmvDFZg7j0,C'est l'un des deux...,Alice sort de prison après avoir purgé une pei...,[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\une chan...,AVI (Microsoft),NaN,Fichier multimédia,2.00 GB,movie/49938,119247.0
2,3709,Un moment d'égarement,NaN,1977,81,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=_HxcgcWv3Hg,NaN,Deux pères emmènent leurs filles âgées de 17 a...,[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\Un momen...,AVI (Microsoft),NaN,Fichier multimédia,696 MB,movie/74871,76849.0
3,3708,Un ange de trop,Heart Condition (Titre original),1990,100,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=U5_C6IrxLWY,A tough cop. A dead lawyer. Every partnership ...,"Se remettant d'un arrêt cardiaque, le détectiv...",[My Book]F:\#DIVX\DIVX MOVIEZ\MadMov\Un Ange D...,Matroska (MKV),NaN,Fichier multimédia,1.91 GB,movie/41817,99750.0
4,3707,Les Guetteurs,The Watchers (Titre original),2024,102,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=YWdWv2iuo8Q,Laissez-les vous voir,Cette forêt n'existe sur aucune carte. Toutes ...,[My Book]F:\#DIVX\DIVX MOVIEZ\MadMov\The Watch...,Matroska (MKV),NaN,Fichier multimédia,4.59 GB,movie/1086747,26736843.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3706,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3707,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3708,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3709,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [54]:
display(df_intermediaire_movie_studio)

,id,Numéro
0,0,3711
1,0,3682
2,0,3676
3,0,3610
4,0,3370
...,...,...
3709,994,182
3710,995,172
3711,996,169
3712,997,108


In [55]:
# Trouver l'index de la ligne avec le 'Studio' NULL
null_studio_index = df_studio[df_studio['Studio'].isnull()].index

# Si une ligne avec 'Studio' NULL est trouvée
if not null_studio_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_studio.loc[null_studio_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_studio['id'].max()
    if max_id > 0:
        df_studio.loc[df_studio.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_studio.head())

# Enregistrer le DataFrame modifié dans la table 'studio' de la base de données
df_studio.to_sql("studio", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_studio = df_studio.merge(df_studio_movie, on="Studio").drop(columns="Studio")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_studio.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_studio.to_sql("movie_studio", index=False, con=engine, if_exists="replace")


    id                   Studio
0  999      Metro-Goldwyn-Mayer
1    1     TF1 Films Production
2    2         Renn Productions
3    3          New Line Cinema
4    4  Jerry Bruckheimer Films
    id  Numéro
0  999    3711
1  999    3682
2  999    3676
3  999    3610
4  999    3370


3714

In [56]:
# df_studio.to_sql("studio",index=False,con = engine,if_exists = "replace" )

In [57]:
df_intermediaire_movie_studio.to_sql("movie_studio",index=False,con = engine,if_exists = "replace" )

3714

# table movie, casting & intermediaire

In [58]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [59]:
list_casting_movie = ["Distribution", 'Numéro']
df_casting_movie = df.copy()[list_casting_movie]

In [60]:
df_casting_movie ["Distribution"] = df_casting_movie["Distribution"].str.split(",\s*")

In [61]:
df_casting_movie  = df_casting_movie.explode("Distribution")

In [62]:
df_casting  = df_casting_movie.explode("Distribution").drop_duplicates("Distribution").reset_index(drop=True).reset_index()
df_casting = df_casting.rename(columns={"index":"id"}).drop(columns="Numéro")

In [63]:
df_intermediaire_movie_casting = df_casting.merge(df_casting_movie, on = "Distribution").drop(columns="Distribution")

In [64]:
display(df_casting)

,id,Distribution
0,0,Judy Garland
1,1,James Stewart
2,2,Hedy Lamarr
3,3,Lana Turner
4,4,Jackie Cooper
...,...,...
10123,10123,Paulette Goddard
10124,10124,Henry Bergman
10125,10125,Tiny Sandford
10126,10126,Jackie Coogan


In [65]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3711,La Danseuse des Folies Ziegfeld (vostfr),Ziegfeld Girl (Titre original),1941,132,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=bRYo5Tsj5ok,GREATEST MUSICAL EXTRAVAGANZA OF ALL TIME!,Trois jeunes filles en quête de gloire sont en...,[My Book]F:\#DIVX\DIVX MOVIEZ\MUSICAL\Ziegfeld...,Matroska (MKV),NaN,Fichier multimédia,1.60 GB,movie/32619,34415.0
1,3710,Une chance sur deux,NaN,1998,110,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=QWmvDFZg7j0,C'est l'un des deux...,Alice sort de prison après avoir purgé une pei...,[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\une chan...,AVI (Microsoft),NaN,Fichier multimédia,2.00 GB,movie/49938,119247.0
2,3709,Un moment d'égarement,NaN,1977,81,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=_HxcgcWv3Hg,NaN,Deux pères emmènent leurs filles âgées de 17 a...,[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\Un momen...,AVI (Microsoft),NaN,Fichier multimédia,696 MB,movie/74871,76849.0
3,3708,Un ange de trop,Heart Condition (Titre original),1990,100,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=U5_C6IrxLWY,A tough cop. A dead lawyer. Every partnership ...,"Se remettant d'un arrêt cardiaque, le détectiv...",[My Book]F:\#DIVX\DIVX MOVIEZ\MadMov\Un Ange D...,Matroska (MKV),NaN,Fichier multimédia,1.91 GB,movie/41817,99750.0
4,3707,Les Guetteurs,The Watchers (Titre original),2024,102,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=YWdWv2iuo8Q,Laissez-les vous voir,Cette forêt n'existe sur aucune carte. Toutes ...,[My Book]F:\#DIVX\DIVX MOVIEZ\MadMov\The Watch...,Matroska (MKV),NaN,Fichier multimédia,4.59 GB,movie/1086747,26736843.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3706,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3707,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3708,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3709,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [66]:
display(df_intermediaire_movie_casting)

,id,Numéro
0,0,3711
1,0,2499
2,1,3711
3,1,3369
4,1,2471
...,...,...
17624,10123,2
17625,10124,2
17626,10125,2
17627,10126,1


In [67]:
# Trouver l'index de la ligne avec la 'Distribution' NULL
null_casting_index = df_casting[df_casting['Distribution'].isnull()].index

# Si une ligne avec 'Distribution' NULL est trouvée
if not null_casting_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_casting.loc[null_casting_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_casting['id'].max()
    if max_id > 0:
        df_casting.loc[df_casting.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_casting.head())

# Enregistrer le DataFrame modifié dans la table 'casting' de la base de données
df_casting.to_sql("casting", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_casting = df_casting.merge(df_casting_movie, on="Distribution").drop(columns="Distribution")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_casting.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_casting.to_sql("movie_casting", index=False, con=engine, if_exists="replace")


      id   Distribution
0  10128   Judy Garland
1      1  James Stewart
2      2    Hedy Lamarr
3      3    Lana Turner
4      4  Jackie Cooper
      id  Numéro
0  10128    3711
1  10128    2499
2      1    3711
3      1    3369
4      1    2471


17629

In [68]:
# df_casting.to_sql("casting",index=False,con = engine,if_exists = "replace" )

In [69]:
# df_intermediaire_movie_casting.to_sql("movie_casting",index=False,con = engine,if_exists = "replace" )

# table movie, country & intermediaire

In [70]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [71]:
list_country_movie = ["Pays", 'Numéro']
df_country_movie = df.copy()[list_country_movie]

In [72]:
df_country_movie ["Pays"] = df_country_movie["Pays"].str.split(",\s*")

In [73]:
df_country_movie  = df_country_movie.explode("Pays")

In [74]:
df_country  = df_country_movie.explode("Pays").drop_duplicates("Pays").reset_index(drop=True).reset_index()
df_country = df_country.rename(columns={"index":"id"}).drop(columns="Numéro")

In [75]:
df_intermediaire_movie_country = df_country.merge(df_country_movie, on = "Pays").drop(columns="Pays")

In [76]:
display(df_country)

,id,Pays
0,0,Etats-Unis
1,1,France
2,2,Royaume-Uni
3,3,Japon
4,4,Corée du Sud
5,5,Inde
6,6,Italie
7,7,Canada
8,8,Belgique
9,9,Pays-Bas


In [77]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3711,La Danseuse des Folies Ziegfeld (vostfr),Ziegfeld Girl (Titre original),1941,132,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=bRYo5Tsj5ok,GREATEST MUSICAL EXTRAVAGANZA OF ALL TIME!,Trois jeunes filles en quête de gloire sont en...,[My Book]F:\#DIVX\DIVX MOVIEZ\MUSICAL\Ziegfeld...,Matroska (MKV),NaN,Fichier multimédia,1.60 GB,movie/32619,34415.0
1,3710,Une chance sur deux,NaN,1998,110,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=QWmvDFZg7j0,C'est l'un des deux...,Alice sort de prison après avoir purgé une pei...,[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\une chan...,AVI (Microsoft),NaN,Fichier multimédia,2.00 GB,movie/49938,119247.0
2,3709,Un moment d'égarement,NaN,1977,81,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=_HxcgcWv3Hg,NaN,Deux pères emmènent leurs filles âgées de 17 a...,[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\Un momen...,AVI (Microsoft),NaN,Fichier multimédia,696 MB,movie/74871,76849.0
3,3708,Un ange de trop,Heart Condition (Titre original),1990,100,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=U5_C6IrxLWY,A tough cop. A dead lawyer. Every partnership ...,"Se remettant d'un arrêt cardiaque, le détectiv...",[My Book]F:\#DIVX\DIVX MOVIEZ\MadMov\Un Ange D...,Matroska (MKV),NaN,Fichier multimédia,1.91 GB,movie/41817,99750.0
4,3707,Les Guetteurs,The Watchers (Titre original),2024,102,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=YWdWv2iuo8Q,Laissez-les vous voir,Cette forêt n'existe sur aucune carte. Toutes ...,[My Book]F:\#DIVX\DIVX MOVIEZ\MadMov\The Watch...,Matroska (MKV),NaN,Fichier multimédia,4.59 GB,movie/1086747,26736843.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3706,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3707,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3708,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3709,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [78]:
display(df_intermediaire_movie_country)

,id,Numéro
0,0,3711
1,0,3708
2,0,3707
3,0,3706
4,0,3705
...,...,...
3706,42,853
3707,43,921
3708,44,769
3709,45,631


In [79]:
# Trouver l'index de la ligne avec le 'Pays' NULL
null_country_index = df_country[df_country['Pays'].isnull()].index

# Si une ligne avec 'Pays' NULL est trouvée
if not null_country_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_country.loc[null_country_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_country['id'].max()
    if max_id > 0:
        df_country.loc[df_country.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_country.head())

# Enregistrer le DataFrame modifié dans la table 'country' de la base de données
df_country.to_sql("country", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_country = df_country.merge(df_country_movie, on="Pays").drop(columns="Pays")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_country.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_country.to_sql("movie_country", index=False, con=engine, if_exists="replace")

   id          Pays
0  46    Etats-Unis
1   1        France
2   2   Royaume-Uni
3   3         Japon
4   4  Corée du Sud
   id  Numéro
0  46    3711
1  46    3708
2  46    3707
3  46    3706
4  46    3705


3711

In [80]:
# df_country.to_sql("country",index=False,con = engine,if_exists = "replace" )

In [81]:
df_intermediaire_movie_country.to_sql("movie_country",index=False,con = engine,if_exists = "replace" )

3711

# table movie, genre & intermediaire

In [82]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [83]:
list_genre_movie = ["Genre(s)", 'Numéro']
df_genre_movie = df.copy()[list_genre_movie]

In [84]:
df_genre_movie ["Genre(s)"] = df_genre_movie["Genre(s)"].str.split(",\s*")

In [85]:
df_genre_movie  = df_genre_movie.explode("Genre(s)")

In [86]:
df_genre  = df_genre_movie.explode("Genre(s)").drop_duplicates("Genre(s)").reset_index(drop=True).reset_index()
df_genre = df_genre.rename(columns={"index":"id"}).drop(columns="Numéro")

In [87]:
df_intermediaire_movie_genre = df_genre.merge(df_genre_movie, on = "Genre(s)").drop(columns="Genre(s)")

In [88]:
display(df_genre)

,id,Genre(s)
0,0,Drame
1,1,Comédie
2,2,Romantique
3,3,Musical
4,4,Action
5,5,Fantastique
6,6,Policier
7,7,Horreur
8,8,Mystère
9,9,Suspense


In [89]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3711,La Danseuse des Folies Ziegfeld (vostfr),Ziegfeld Girl (Titre original),1941,132,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=bRYo5Tsj5ok,GREATEST MUSICAL EXTRAVAGANZA OF ALL TIME!,Trois jeunes filles en quête de gloire sont en...,[My Book]F:\#DIVX\DIVX MOVIEZ\MUSICAL\Ziegfeld...,Matroska (MKV),NaN,Fichier multimédia,1.60 GB,movie/32619,34415.0
1,3710,Une chance sur deux,NaN,1998,110,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=QWmvDFZg7j0,C'est l'un des deux...,Alice sort de prison après avoir purgé une pei...,[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\une chan...,AVI (Microsoft),NaN,Fichier multimédia,2.00 GB,movie/49938,119247.0
2,3709,Un moment d'égarement,NaN,1977,81,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=_HxcgcWv3Hg,NaN,Deux pères emmènent leurs filles âgées de 17 a...,[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\Un momen...,AVI (Microsoft),NaN,Fichier multimédia,696 MB,movie/74871,76849.0
3,3708,Un ange de trop,Heart Condition (Titre original),1990,100,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=U5_C6IrxLWY,A tough cop. A dead lawyer. Every partnership ...,"Se remettant d'un arrêt cardiaque, le détectiv...",[My Book]F:\#DIVX\DIVX MOVIEZ\MadMov\Un Ange D...,Matroska (MKV),NaN,Fichier multimédia,1.91 GB,movie/41817,99750.0
4,3707,Les Guetteurs,The Watchers (Titre original),2024,102,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=YWdWv2iuo8Q,Laissez-les vous voir,Cette forêt n'existe sur aucune carte. Toutes ...,[My Book]F:\#DIVX\DIVX MOVIEZ\MadMov\The Watch...,Matroska (MKV),NaN,Fichier multimédia,4.59 GB,movie/1086747,26736843.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3706,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3707,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3708,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3709,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [90]:
display(df_intermediaire_movie_genre)

,id,Numéro
0,0,3711
1,0,3709
2,0,3704
3,0,3703
4,0,3698
...,...,...
8156,37,36
8157,37,35
8158,38,3113
8159,38,389


In [91]:
# Trouver l'index de la ligne avec le 'Genre(s)' NULL
null_genre_index = df_genre[df_genre['Genre(s)'].isnull()].index

# Si une ligne avec 'Genre(s)' NULL est trouvée
if not null_genre_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_genre.loc[null_genre_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_genre['id'].max()
    if max_id > 0:
        df_genre.loc[df_genre.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_genre.head())

# Enregistrer le DataFrame modifié dans la table 'genre' de la base de données
df_genre.to_sql("genre", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_genre = df_genre.merge(df_genre_movie, on="Genre(s)").drop(columns="Genre(s)")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_genre.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_genre.to_sql("movie_genre", index=False, con=engine, if_exists="replace")


   id    Genre(s)
0  38       Drame
1   1     Comédie
2   2  Romantique
3   3     Musical
4   4      Action
   id  Numéro
0  38    3711
1  38    3709
2  38    3704
3  38    3703
4  38    3698


8161

In [92]:
df_genre.to_sql("genre",index=False,con = engine,if_exists = "replace" )

39

In [93]:
df_intermediaire_movie_genre.to_sql("movie_genre",index=False,con = engine,if_exists = "replace" )

8161

# table movie, language & intermediaire

In [94]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [95]:
list_language_movie = ["Langues", 'Numéro']
df_language_movie = df.copy()[list_language_movie]

In [96]:
df_language_movie ["Langues"] = df_language_movie["Langues"].str.split(",\s*")

In [97]:
df_language_movie  = df_language_movie.explode("Langues")

In [98]:
df_language  = df_language_movie.explode("Langues").drop_duplicates("Langues").reset_index(drop=True).reset_index()
df_language = df_language.rename(columns={"index":"id"}).drop(columns="Numéro")

In [99]:
df_intermediaire_movie_language = df_language.merge(df_language_movie, on = "Langues").drop(columns="Langues")

In [100]:
display(df_language)

,id,Langues
0,0,Anglais
1,1,Français
2,2,Allemand
3,3,Japonais
4,4,NaN
5,5,Coréen
6,6,Italien
7,7,Espagnol
8,8,Néerlandais
9,9,Polonais


In [101]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3711,La Danseuse des Folies Ziegfeld (vostfr),Ziegfeld Girl (Titre original),1941,132,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=bRYo5Tsj5ok,GREATEST MUSICAL EXTRAVAGANZA OF ALL TIME!,Trois jeunes filles en quête de gloire sont en...,[My Book]F:\#DIVX\DIVX MOVIEZ\MUSICAL\Ziegfeld...,Matroska (MKV),NaN,Fichier multimédia,1.60 GB,movie/32619,34415.0
1,3710,Une chance sur deux,NaN,1998,110,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=QWmvDFZg7j0,C'est l'un des deux...,Alice sort de prison après avoir purgé une pei...,[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\une chan...,AVI (Microsoft),NaN,Fichier multimédia,2.00 GB,movie/49938,119247.0
2,3709,Un moment d'égarement,NaN,1977,81,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=_HxcgcWv3Hg,NaN,Deux pères emmènent leurs filles âgées de 17 a...,[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\Un momen...,AVI (Microsoft),NaN,Fichier multimédia,696 MB,movie/74871,76849.0
3,3708,Un ange de trop,Heart Condition (Titre original),1990,100,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=U5_C6IrxLWY,A tough cop. A dead lawyer. Every partnership ...,"Se remettant d'un arrêt cardiaque, le détectiv...",[My Book]F:\#DIVX\DIVX MOVIEZ\MadMov\Un Ange D...,Matroska (MKV),NaN,Fichier multimédia,1.91 GB,movie/41817,99750.0
4,3707,Les Guetteurs,The Watchers (Titre original),2024,102,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=YWdWv2iuo8Q,Laissez-les vous voir,Cette forêt n'existe sur aucune carte. Toutes ...,[My Book]F:\#DIVX\DIVX MOVIEZ\MadMov\The Watch...,Matroska (MKV),NaN,Fichier multimédia,4.59 GB,movie/1086747,26736843.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3706,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3707,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3708,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3709,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [102]:
display(df_intermediaire_movie_language)

,id,Numéro
0,0,3711
1,0,3708
2,0,3707
3,0,3706
4,0,3705
...,...,...
5214,42,1060
5215,43,1053
5216,43,930
5217,43,926


In [103]:
# Trouver l'index de la ligne avec les 'Langues' NULL
null_language_index = df_language[df_language['Langues'].isnull()].index

# Si une ligne avec 'Langues' NULL est trouvée
if not null_language_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_language.loc[null_language_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_language['id'].max()
    if max_id > 0:
        df_language.loc[df_language.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_language.head())

# Enregistrer le DataFrame modifié dans la table 'language' de la base de données
df_language.to_sql("language", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_language = df_language.merge(df_language_movie, on="Langues").drop(columns="Langues")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_language.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_language.to_sql("movie_language", index=False, con=engine, if_exists="replace")


   id   Langues
0  45   Anglais
1   1  Français
2   2  Allemand
3   3  Japonais
4   0       NaN
   id  Numéro
0  45    3711
1  45    3708
2  45    3707
3  45    3706
4  45    3705


5219

In [104]:
# df_language.to_sql("language",index=False,con = engine,if_exists = "replace" )

In [105]:
df_intermediaire_movie_language.to_sql("movie_language",index=False,con = engine,if_exists = "replace" )

5219

# table, movie, tag & intermediaire

In [106]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [107]:
list_tag_movie = ["Etiquettes", 'Numéro']
df_tag_movie = df.copy()[list_tag_movie]

In [108]:
df_tag_movie ["Etiquettes"] = df_tag_movie["Etiquettes"].str.split(",\s*")

In [109]:
df_tag_movie  = df_tag_movie.explode("Etiquettes")

In [110]:
df_tag = df_tag_movie.explode("Etiquettes").drop_duplicates("Etiquettes").reset_index(drop=True).reset_index()
df_tag = df_tag.rename(columns={"index":"id"}).drop(columns="Numéro")

In [111]:
df_intermediaire_movie_tag = df_tag.merge(df_tag_movie, on = "Etiquettes").drop(columns="Etiquettes")

In [112]:
display(df_tag)

,id,Etiquettes
0,0,NaN
1,1,Ange
2,2,Fantômes
3,3,Meta-Film
4,4,Cape et d'épée
...,...,...
203,203,Momie
204,204,viking
205,205,Réalisme soviétique
206,206,Guy Debord


In [113]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3711,La Danseuse des Folies Ziegfeld (vostfr),Ziegfeld Girl (Titre original),1941,132,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=bRYo5Tsj5ok,GREATEST MUSICAL EXTRAVAGANZA OF ALL TIME!,Trois jeunes filles en quête de gloire sont en...,[My Book]F:\#DIVX\DIVX MOVIEZ\MUSICAL\Ziegfeld...,Matroska (MKV),NaN,Fichier multimédia,1.60 GB,movie/32619,34415.0
1,3710,Une chance sur deux,NaN,1998,110,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=QWmvDFZg7j0,C'est l'un des deux...,Alice sort de prison après avoir purgé une pei...,[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\une chan...,AVI (Microsoft),NaN,Fichier multimédia,2.00 GB,movie/49938,119247.0
2,3709,Un moment d'égarement,NaN,1977,81,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=_HxcgcWv3Hg,NaN,Deux pères emmènent leurs filles âgées de 17 a...,[My Book]F:\#DIVX\DIVX MOVIEZ\COMEDIE\Un momen...,AVI (Microsoft),NaN,Fichier multimédia,696 MB,movie/74871,76849.0
3,3708,Un ange de trop,Heart Condition (Titre original),1990,100,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=U5_C6IrxLWY,A tough cop. A dead lawyer. Every partnership ...,"Se remettant d'un arrêt cardiaque, le détectiv...",[My Book]F:\#DIVX\DIVX MOVIEZ\MadMov\Un Ange D...,Matroska (MKV),NaN,Fichier multimédia,1.91 GB,movie/41817,99750.0
4,3707,Les Guetteurs,The Watchers (Titre original),2024,102,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=YWdWv2iuo8Q,Laissez-les vous voir,Cette forêt n'existe sur aucune carte. Toutes ...,[My Book]F:\#DIVX\DIVX MOVIEZ\MadMov\The Watch...,Matroska (MKV),NaN,Fichier multimédia,4.59 GB,movie/1086747,26736843.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3706,5,le mécano de la général,The General (Titre original),1926,79,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=-Dcxawya67g,"Buster drives """"The General"""" to trainload of ...","Conducteur de locomotive, Johnnie Gray habite ...",NaN,Inconnu,NaN,DVD original,NaN,movie/961,17925.0
3707,4,Charles Chaplin : Courts métrages First National,NaN,1918-1922,302,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,disc 1 : Une journée de plaisir (1919) / Une i...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3708,3,Les feux de la rampe,Limelight (Titre original),1952,137,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=mme-SQcu8YE,The masterpiece of laughter and tears from the...,Un comique du music-hall déchu se prend d'affe...,NaN,Inconnu,NaN,DVD original,NaN,movie/28971,44837.0
3709,2,Les temps modernes,Modern Times (Titre original),1936,87,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=kNIh3C1k4aM,He stands alone as the greatest entertainer of...,Charlot est ouvrier dans une gigantesque usine...,NaN,Inconnu,NaN,DVD original,NaN,movie/3082,27977.0


In [114]:
display(df_intermediaire_movie_tag)

,id,Numéro
0,0,3711
1,0,3710
2,0,3709
3,0,3707
4,0,3706
...,...,...
4668,206,74
4669,206,73
4670,207,17
4671,207,16


In [115]:
# Trouver l'index de la ligne avec les 'Etiquettes' NULL
null_tag_index = df_tag[df_tag['Etiquettes'].isnull()].index

# Si une ligne avec 'Etiquettes' NULL est trouvée
if not null_tag_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_tag.loc[null_tag_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_tag['id'].max()
    if max_id > 0:
        df_tag.loc[df_tag.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_tag.head())

# Enregistrer le DataFrame modifié dans la table 'tag' de la base de données
df_tag.to_sql("tag", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_tag = df_tag.merge(df_tag_movie, on="Etiquettes").drop(columns="Etiquettes")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_tag.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_tag.to_sql("movie_tag", index=False, con=engine, if_exists="replace")


    id      Etiquettes
0  208             NaN
1    1            Ange
2    2        Fantômes
3    3       Meta-Film
4    4  Cape et d'épée
    id  Numéro
0  208    3711
1  208    3710
2  208    3709
3  208    3707
4  208    3706


4673

In [116]:
# df_tag.to_sql("tag",index=False,con = engine,if_exists = "replace" )

In [117]:
# df_intermediaire_movie_tag.to_sql("movie_tag",index=False,con = engine,if_exists = "replace" )